# Problema de dimensionamento de lotes

In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Descrição do problema

Uma fábrica precisa encontrar um planejamento de produção de um determinado item para atende demandas em um horizonte de n períodos.

Em cada período $t$ existe uma demanda $d_t$ que pode ser atendida por produção no período $t$ ou por itens em estoque.

Suponha que no início do processo de produção o estoque inicial seja igual a zero, assim como no útimo período.

Considere os seguintes dados para o problema.

- $f_t$, custo fixo de produção no período $t$
- $p_t$, custo de produção unitário no perído $t$
- $h_t$, custo de estoque unitário no período $t$
- $d_t$, quantidade da demanta no período $t$

## Formulação do problema

1. Variáveis

    - $x_i$, quantidade produzida no período $t$
    - $s_t$, quantidade em estoque no final do período $t$
    - $ y_t = 
    \begin{cases}
    1, & \text{se ocorre produção no período {\it t}} \\
    0, & \text{caso contrário}
    \end{cases}
    $

2. Restrições
    - fluxo de produção
    $
    \begin{align}\notag
    s_{t-1} + x_t & = d_t + s_t \text{ para } t=1, 2, \ldots, n
    \end{align}
    $

    - limitação de produção em $t$
    $ 
    \begin{align}\notag
    x_t & \leq (\sum_{i=t}^{n} d_i) y_t, \text{ para } t=1,2, \ldots, n
    \end{align}
    $

    - estoque inicial e final igual a zero
    $$
    s_0 = s_n = 0
    $$

    - não negatividade e integralidade
    $
    \begin{align}
    x_t & \geq 0, \text{ para } t=1,2, \ldots, n \\
    s_t & \geq 0, \text{ para } t=1,2, \ldots, n \\
    y_t & \in \{0,1\}, \text{ para } t=1,2, \ldots, n \\
    \end{align}
    $

    - função objetivo
    $$
    \min \sum_{t=1}^{n} p_t x_t + \sum_{t=1}^{n} h_t s_t + \sum_{t=1}^{n} f_t y_t
    $$

In [ ]:
def readdata(datafile):
  
    with open(datafile, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # lendo o tamanho da instancia
    aux = 0
    N = int(linhas[aux])
  
    # definindo vetores
    H = np.zeros(N)
    P = np.zeros(N)
    F = np.zeros(N)
    D = np.zeros(N)
  
    # lendo e armazenando dados
    aux = aux+1
    F[0] = float(linhas[aux])
    for i in range(1,N):
        F[i] = F[0]

    aux = aux+1
    H[0] = float(linhas[aux])
    for i in range(1,N):
        H[i] = H[0]

    aux = aux+1
    P[0] = float(linhas[aux])
    for i in range(1,N):
        P[i] = P[0]

    aux = aux+1
    D = np.fromstring(linhas[aux], dtype=float, sep = ' ')
 
    return N, H, P, F, D

In [ ]:
def modelo(N, H, P, F, D, model):

    # adicionando variaveis
    x = model.addVars(N, name='x') 
    s = model.addVars(N, name='s')  
    y = model.addVars(N, vtype=GRB.BINARY, name='y') 

    # definindo funcao objetivo
    obj = 0
    for i in range(0, N):
        obj += P[i] * x[i]
        obj += H[i] * s[i]
        obj += F[i] * y[i]

    model.setObjective(obj, GRB.MINIMIZE)
  
    # definindo restrições
    model.addConstr(x[0] - s[0] == D[0])
    for i in range(1, N):
        model.addConstr(s[i-1] + x[i] - s[i] == D[i])
  
    for i in range(0, N):
        model.addConstr(x[i] - (D[i:N].sum())*y[i] <= 0)

    model.addConstr(s[N-1] == 0)

    # exportando modelo
    model.write("uls.lp")

    return model

In [ ]:
def solve(model):
    # resolvendo o problema
    model.optimize()

In [ ]:
if __name__ == "__main__":
    
    datafile = "../data/uls/52_2.txt"

    # lendo os dados
    N, H, P, F, D = readdata(datafile)

    # criando o modelo
    model = gp.Model("uls")

    # configurando parametros
    #model.Params.IterationLimit = 1000 # define o número de iterações do simplex
    model.Params.TimeLimit = 120 # define tempo limite
    model.Params.method = 0 #-1=automatic, 0=primal, 1=dual , 2=barrier
    #model.Params.NodeMethod = -1 #-1=automatic, 0=primal, 1=dual , 2=barrier
    model.Params.Threads = 1

    # modelando o problema
    modelo(N, H, P, F, D, model)

    # resolvendo o problema
    solve(model)
    

In [ ]:
# imprimindo solucao
for var in model.getVars():
    if var.x > 1e-6:
        print(f"{var.varName} = {var.x}")

In [ ]:
# imprimindo o valor otimo
print(f"Objetivo = {model.objVal}")

In [ ]:
# imprime tempo de execucao
print(f'Runtime: {model.Runtime}')

In [ ]:
# clear model
model.dispose()